<a href="https://colab.research.google.com/github/jadenfang/NextWordPredictor/blob/main/Next_Word_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading AI model...


In [36]:
# Simple example
text = "The moon was shining bright at night, contrasting the dark sky"
print(f"\nStarting text: '{text}'")
print("\nGenerating word by word...\n")

# Generate 5 words, one at a time
current_text = text
for step in range(30):
    print(f"--- Step {step + 1} ---")
    print(f"Current: '{current_text}'")

    # Encode current text
    input_ids = tokenizer.encode(current_text, return_tensors="pt")

    # Get predictions
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits

    # Get the predictions for the NEXT token
    next_token_logits = predictions[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim=0)

    # Get top 3 predictions
    top_probs, top_indices = torch.topk(next_token_probs, 10)
    top_probs = top_probs / top_probs.sum()  # renormalize

    print("Top 3 next word predictions:")
    for i, (prob, idx) in enumerate(zip(top_probs, top_indices)):
        word = tokenizer.decode([idx])
        print(f"  {i+1}. '{word}' ({prob.item()*100:.1f}% confident)")

    # Use the most likely word
    next_token_id = top_indices[torch.multinomial(top_probs, 1).item()]
    next_word = tokenizer.decode([next_token_id])
    current_text += next_word


    print(f"✓ Chosen: '{next_word}'")
    print(f"New text: '{current_text}'\n")

print(f"\nFinal generated text: '{current_text}'")


Starting text: 'The moon was shining bright at night, contrasting the dark sky'

Generating word by word...

--- Step 1 ---
Current: 'The moon was shining bright at night, contrasting the dark sky'
Top 3 next word predictions:
  1. ' with' (88.8% confident)
  2. ' and' (3.8% confident)
  3. '.' (1.3% confident)
  4. ' of' (1.2% confident)
  5. ' to' (0.9% confident)
  6. ' for' (0.9% confident)
  7. ' in' (0.8% confident)
  8. ''s' (0.8% confident)
  9. ',' (0.8% confident)
  10. ' from' (0.7% confident)
✓ Chosen: ' with'
New text: 'The moon was shining bright at night, contrasting the dark sky with'

--- Step 2 ---
Current: 'The moon was shining bright at night, contrasting the dark sky with'
Top 3 next word predictions:
  1. ' the' (79.1% confident)
  2. ' a' (9.2% confident)
  3. ' light' (2.9% confident)
  4. ' bright' (2.5% confident)
  5. ' white' (1.5% confident)
  6. ' its' (1.3% confident)
  7. ' an' (1.2% confident)
  8. ' dark' (1.0% confident)
  9. ' moon' (0.7% confident)